In [1]:
import glob
import re

import numpy as np
import pandas as pd
import tqdm

pd.set_option("display.max_columns", None)


In [2]:
def join_games(file_list):
   
   df = pd.DataFrame()

   for match in tqdm.tqdm(file_list, ncols=100):

      clean_match = match.split("\\")
      # print(' '.join(clean_match))

      year = int(clean_match[0].split("/")[1])
      jornada = int(clean_match[1].split()[-1])
      local = clean_match[-1].split(" vs ")[0]
      visit = clean_match[-1].split(" vs ")[1].replace(".xlsx", "")
      game = clean_match[-1].replace(".xlsx", "")
      # print(year, jornada, local, visit)

      df_local = pd.read_excel(match, sheet_name=local)
      df_local["Equipo"] = local
      df_local["Estado"] = 'Local'
      df_local["Siglas"] = re.sub("[^A-Z]", "", local)
      df_local["Jornada"] = jornada
      df_local["Anio"] = year
      df_local["Game"] = game

      df_visit = pd.read_excel(match, sheet_name=visit)
      df_visit["Equipo"] = visit
      df_visit["Estado"] = 'Visitante'
      df_visit["Siglas"] = re.sub("[^A-Z]", "", visit)
      df_visit["Jornada"] = jornada
      df_visit["Anio"] = year
      df_visit["Game"] = game

      df = pd.concat([df, df_local, df_visit], ignore_index=True)

   df.columns = ['Jugador', 'P', 'M', 'V', 'AG', 'AST', 'A', 'R', 'PENALTI', 'PP', 'PF',
                    'D', 'DP', 'D1', 'DS', 'Rob', 'P1', 'PC', 'PPC', 'D2', 'FC', 'FS',
                    'Equipo', 'Estado', 'Siglas', 'Jornada', 'Anio', 'Game']

   df.PPC = df.PPC.replace({'%':''}, regex=True)
   num_columns = ['M', 'V', 'AG', 'AST', 'A', 'R', 'PENALTI', 'PP', 'PF',
       'D', 'DP', 'D1', 'DS', 'Rob', 'P1', 'PC', 'PPC', 'D2', 'FC', 'FS']

   df[num_columns] = df[num_columns].apply(pd.to_numeric, errors='coerce', axis=1)
    
   return df


In [3]:
excels_2020 = glob.glob("data/2020/*/*.xlsx")
excels_2021 = glob.glob("data/2021/*/*.xlsx")


In [4]:
df_2020 = join_games(excels_2020)
df_2021 = join_games(excels_2021)

100%|█████████████████████████████████████████████████████████████| 153/153 [00:04<00:00, 35.78it/s]


# EDA 2020

In [5]:
df_2020.head()

,Jugador,P,M,V,AG,AST,A,R,PENALTI,PP,PF,D,DP,D1,DS,Rob,P1,PC,PPC,D2,FC,FS,Equipo,Estado,Siglas,Jornada,Anio,Game
0,Érick Delgado,P,85.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19.0,7.0,37.0,3.0,0.0,1.0,Academia Cantolao,Local,AC,1,2020,Academia Cantolao vs Cienciano
1,José Ramírez,D,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,33.0,28.0,85.0,0.0,0.0,0.0,Academia Cantolao,Local,AC,1,2020,Academia Cantolao vs Cienciano
2,Orlando Núñez,D,43.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,8.0,89.0,0.0,3.0,2.0,Academia Cantolao,Local,AC,1,2020,Academia Cantolao vs Cienciano
3,Arón Sánchez,D,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,16.0,33.0,30.0,91.0,0.0,0.0,2.0,Academia Cantolao,Local,AC,1,2020,Academia Cantolao vs Cienciano
4,Christian Sánchez,D,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,5.0,24.0,17.0,71.0,0.0,0.0,0.0,Academia Cantolao,Local,AC,1,2020,Academia Cantolao vs Cienciano


In [6]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6781 entries, 0 to 6780
Data columns (total 28 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Jugador  6781 non-null   object 
 1   P        6781 non-null   object 
 2   M        6781 non-null   float64
 3   V        6781 non-null   float64
 4   AG       6781 non-null   float64
 5   AST      6781 non-null   float64
 6   A        6781 non-null   float64
 7   R        6781 non-null   float64
 8   PENALTI  6781 non-null   float64
 9   PP       6781 non-null   float64
 10  PF       6781 non-null   float64
 11  D        6277 non-null   float64
 12  DP       6277 non-null   float64
 13  D1       6277 non-null   float64
 14  DS       6277 non-null   float64
 15  Rob      6277 non-null   float64
 16  P1       6277 non-null   float64
 17  PC       6277 non-null   float64
 18  PPC      6277 non-null   float64
 19  D2       6277 non-null   float64
 20  FC       6277 non-null   float64
 21  FS       6277 

## Eliminando partidos con registros nulos

In [7]:
nan_rows = df_2020[df_2020.isnull().any(axis=1)].copy().reset_index(drop=True)
# nan_jornada_games_2020 = nan_rows.groupby(['Jornada','Game']).size().reset_index(name='Cantidad')
# print(nan_jornada_games_2020)
drop_games_2020 = nan_rows.Game.unique()
drop_games_2020
# nan_rows.Game.unique()

array(['FC Carlos Stein vs Carlos A. Mannucci',
       'Alianza Huánuco vs Cusco FC',
       'FC Carlos Stein vs Deportivo Llacuabamba',
       'Alianza Huánuco vs FC Carlos Stein',
       'Atlético Grau vs Univ. César Vallejo',
       'Deportivo Llacuabamba vs Academia Cantolao',
       'FC Carlos Stein vs Universitario', 'Alianza Huánuco vs Cienciano',
       'Atlético Grau vs Sport Boys Callao',
       'Deportivo Llacuabamba vs Univ. San Martín',
       'Alianza Huánuco vs Deportivo Llacuabamba',
       'FC Carlos Stein vs Binacional',
       'Atlético Grau vs Univ. San Martín',
       'Cusco FC vs Sport Boys Callao'], dtype=object)

In [8]:
df_2020.columns

Index(['Jugador', 'P', 'M', 'V', 'AG', 'AST', 'A', 'R', 'PENALTI', 'PP', 'PF',
       'D', 'DP', 'D1', 'DS', 'Rob', 'P1', 'PC', 'PPC', 'D2', 'FC', 'FS',
       'Equipo', 'Estado', 'Siglas', 'Jornada', 'Anio', 'Game'],
      dtype='object')

In [9]:
df_2020 = df_2020.loc[~df_2020.Game.isin(drop_games_2020)].reset_index(drop=True)

In [10]:
df_2020.head()

,Jugador,P,M,V,AG,AST,A,R,PENALTI,PP,PF,D,DP,D1,DS,Rob,P1,PC,PPC,D2,FC,FS,Equipo,Estado,Siglas,Jornada,Anio,Game
0,Érick Delgado,P,85.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19.0,7.0,37.0,3.0,0.0,1.0,Academia Cantolao,Local,AC,1,2020,Academia Cantolao vs Cienciano
1,José Ramírez,D,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,33.0,28.0,85.0,0.0,0.0,0.0,Academia Cantolao,Local,AC,1,2020,Academia Cantolao vs Cienciano
2,Orlando Núñez,D,43.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,8.0,89.0,0.0,3.0,2.0,Academia Cantolao,Local,AC,1,2020,Academia Cantolao vs Cienciano
3,Arón Sánchez,D,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,16.0,33.0,30.0,91.0,0.0,0.0,2.0,Academia Cantolao,Local,AC,1,2020,Academia Cantolao vs Cienciano
4,Christian Sánchez,D,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,5.0,24.0,17.0,71.0,0.0,0.0,0.0,Academia Cantolao,Local,AC,1,2020,Academia Cantolao vs Cienciano


In [11]:
df_2020.columns

Index(['Jugador', 'P', 'M', 'V', 'AG', 'AST', 'A', 'R', 'PENALTI', 'PP', 'PF',
       'D', 'DP', 'D1', 'DS', 'Rob', 'P1', 'PC', 'PPC', 'D2', 'FC', 'FS',
       'Equipo', 'Estado', 'Siglas', 'Jornada', 'Anio', 'Game'],
      dtype='object')

In [12]:
sum_columns = ['V', 'AG', 'AST', 'A', 'R', 'PENALTI', 'PP', 'PF',
       'D', 'DP', 'D1', 'DS', 'Rob', 'P1', 'PC', 'D2', 'FC', 'FS']

df_2020_groupby = df_2020.groupby(['Game', 'Jornada','Estado'])[sum_columns].sum().sort_values('Game').reset_index().copy()
df_2020_groupby


,Game,Jornada,Estado,V,AG,AST,A,R,PENALTI,PP,PF,D,DP,D1,DS,Rob,P1,PC,D2,FC,FS
0,Academia Cantolao vs Alianza Huánuco,18,Local,1.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,5.0,1.0,16.0,12.0,14.0,376.0,277.0,7.0,18.0,17.0
1,Academia Cantolao vs Alianza Huánuco,18,Visitante,2.0,0.0,1.0,5.0,0.0,0.0,0.0,1.0,21.0,10.0,18.0,11.0,12.0,365.0,275.0,0.0,18.0,18.0
2,Academia Cantolao vs Alianza Lima,17,Local,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,7.0,1.0,21.0,18.0,12.0,320.0,239.0,2.0,16.0,29.0
3,Academia Cantolao vs Alianza Lima,17,Visitante,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,6.0,2.0,14.0,8.0,6.0,403.0,319.0,0.0,30.0,14.0
4,Academia Cantolao vs Atlético Grau,5,Local,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,13.0,3.0,10.0,7.0,5.0,643.0,550.0,1.0,12.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Universitario vs Sport Huancayo,2,Visitante,1.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,8.0,3.0,27.0,19.0,22.0,400.0,307.0,2.0,17.0,16.0
346,Universitario vs Univ. César Vallejo,4,Local,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,16.0,3.0,26.0,19.0,9.0,397.0,306.0,0.0,22.0,23.0
347,Universitario vs Univ. César Vallejo,4,Visitante,2.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,12.0,2.0,13.0,8.0,20.0,400.0,310.0,3.0,25.0,20.0
348,Universitario vs Univ. San Martín,8,Local,2.0,0.0,1.0,3.0,0.0,1.0,0.0,0.0,8.0,4.0,20.0,14.0,15.0,376.0,276.0,2.0,15.0,13.0


In [13]:
games_sum = [df_2020_groupby.loc[i:i+1, :] for i in range(0, len(df_2020_groupby), 2)]

In [24]:
games_sum = [df_2020_groupby.loc[i:i+1, :] for i in range(0, len(df_2020_groupby), 2)]

df_2020 = pd.DataFrame()
for dataframe in games_sum:
    temp_1 = pd.DataFrame(dataframe.iloc[0]).T.reset_index(drop=True)
    temp_1.drop(columns=['Estado', 'Jornada'], inplace=True)
    temp_1.columns = [col+'_L' for col in temp_1.columns]
    temp_1.rename(columns={'Game_L':'Game'}, inplace=True)

    temp_2 = pd.DataFrame(dataframe.iloc[1]).T.reset_index(drop=True)
    temp_2.drop(columns=['Game','Jornada','Estado'], inplace=True)
    temp_2.columns = [col+'_V' for col in temp_2.columns]

    merge_temp = pd.concat([temp_1, temp_2], axis=1)
    
    df_2020 = pd.concat([df_2020, merge_temp], ignore_index=True)

In [25]:
df_2020.head()

,Game,V_L,AG_L,AST_L,A_L,R_L,PENALTI_L,PP_L,PF_L,D_L,DP_L,D1_L,DS_L,Rob_L,P1_L,PC_L,D2_L,FC_L,FS_L,V_V,AG_V,AST_V,A_V,R_V,PENALTI_V,PP_V,PF_V,D_V,DP_V,D1_V,DS_V,Rob_V,P1_V,PC_V,D2_V,FC_V,FS_V
0,Academia Cantolao vs Alianza Huánuco,1.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,5.0,1.0,16.0,12.0,14.0,376.0,277.0,7.0,18.0,17.0,2.0,0.0,1.0,5.0,0.0,0.0,0.0,1.0,21.0,10.0,18.0,11.0,12.0,365.0,275.0,0.0,18.0,18.0
1,Academia Cantolao vs Alianza Lima,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,7.0,1.0,21.0,18.0,12.0,320.0,239.0,2.0,16.0,29.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,6.0,2.0,14.0,8.0,6.0,403.0,319.0,0.0,30.0,14.0
2,Academia Cantolao vs Atlético Grau,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,13.0,3.0,10.0,7.0,5.0,643.0,550.0,1.0,12.0,12.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0,1.0,16.0,10.0,32.0,239.0,156.0,2.0,14.0,11.0
3,Academia Cantolao vs Ayacucho FC,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,14.0,5.0,12.0,11.0,14.0,435.0,352.0,4.0,16.0,23.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,10.0,5.0,21.0,17.0,16.0,296.0,210.0,5.0,24.0,16.0
4,Academia Cantolao vs Cienciano,1.0,0.0,1.0,4.0,2.0,0.0,1.0,0.0,25.0,5.0,24.0,14.0,7.0,277.0,203.0,5.0,20.0,16.0,2.0,0.0,0.0,8.0,3.0,2.0,0.0,1.0,12.0,7.0,12.0,7.0,48.0,264.0,214.0,3.0,16.0,19.0


In [26]:
games_2020 = pd.read_pickle('data/2020_stats_final.pkl')
result = pd.merge(games_2020, df_2020, on="Game")

In [29]:
result.to_parquet('stats_2020.parquet')

In [30]:
pd.read_parquet('stats_2020.parquet')

,Jornada,Fecha,Local,Visitante,Goles local,Goles visitante,Link,Remates a puerta local,Remates a puerta visitante,Tiros local,Tiros visitante,Fueras de juego local,Fueras de juego visitante,Faltas local,Faltas visitante,Saques de esquina local,Saques de esquina visitante,Posesión local,Posesión visitante,Game,V_L,AG_L,AST_L,A_L,R_L,PENALTI_L,PP_L,PF_L,D_L,DP_L,D1_L,DS_L,Rob_L,P1_L,PC_L,D2_L,FC_L,FS_L,V_V,AG_V,AST_V,A_V,R_V,PENALTI_V,PP_V,PF_V,D_V,DP_V,D1_V,DS_V,Rob_V,P1_V,PC_V,D2_V,FC_V,FS_V
0,1,2020-01-02,FBC Melgar,Universitario,1,2,https://www.ceroacero.es/match.php?id=7324623,7,2,20,7,1,1,15,21,8,3,59 %,41 %,FBC Melgar vs Universitario,2.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,7.0,2.0,14.0,10.0,25.0,326.0,242.0,6.0,21.0,13.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,20.0,7.0,13.0,8.0,8.0,463.0,382.0,0.0,15.0,18.0
1,1,2020-01-02,Alianza Lima,Alianza Huánuco,2,3,https://www.ceroacero.es/match.php?id=7324624,4,5,15,9,None,None,15,9,7,2,54 %,46 %,Alianza Lima vs Alianza Huánuco,2.0,0.0,2.0,5.0,1.0,0.0,0.0,0.0,15.0,4.0,14.0,9.0,6.0,437.0,375.0,2.0,15.0,9.0,3.0,0.0,2.0,5.0,0.0,1.0,0.0,0.0,9.0,5.0,12.0,9.0,19.0,391.0,323.0,2.0,9.0,14.0
2,1,2020-01-02,Univ. Técnica,Sporting Cristal,2,1,https://www.ceroacero.es/match.php?id=7324625,5,6,19,14,4,0,21,8,2,5,39 %,61 %,Univ. Técnica vs Sporting Cristal,2.0,0.0,2.0,2.0,1.0,0.0,0.0,0.0,19.0,5.0,10.0,8.0,13.0,299.0,251.0,5.0,21.0,7.0,1.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,14.0,6.0,12.0,11.0,10.0,479.0,420.0,3.0,8.0,20.0
3,1,2020-01-02,Sport Huancayo,Atlético Grau,1,0,https://www.ceroacero.es/match.php?id=7324626,6,3,24,4,3,2,9,8,13,4,65 %,35 %,Sport Huancayo vs Atlético Grau,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,6.0,10.0,8.0,6.0,572.0,517.0,3.0,9.0,7.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,3.0,14.0,10.0,27.0,316.0,258.0,5.0,8.0,9.0
4,1,2020-02-02,Univ. César Vallejo,Deportivo Municipal,1,1,https://www.ceroacero.es/match.php?id=7324627,6,1,18,5,1,4,13,18,13,1,75 %,25 %,Univ. César Vallejo vs Deportivo Municipal,1.0,0.0,0.0,5.0,0.0,1.0,0.0,0.0,5.0,1.0,13.0,9.0,43.0,189.0,108.0,5.0,18.0,9.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,18.0,6.0,12.0,12.0,5.0,580.0,489.0,0.0,13.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,19,2020-10-19,Ayacucho FC,FC Carlos Stein,3,0,https://www.ceroacero.es/match.php?id=7324816,7,3,16,5,4,0,14,13,4,2,56 %,44 %,Ayacucho FC vs FC Carlos Stein,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,3.0,16.0,11.0,29.0,306.0,227.0,4.0,13.0,14.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,16.0,7.0,14.0,12.0,14.0,384.0,321.0,3.0,14.0,12.0
171,19,2020-10-19,Deportivo Llacuabamba,Univ. César Vallejo,2,3,https://www.ceroacero.es/match.php?id=7324815,4,8,14,13,2,4,13,24,5,5,47 %,53 %,Deportivo Llacuabamba vs Univ. César Vallejo,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,14.0,4.0,10.0,7.0,13.0,465.0,405.0,5.0,13.0,20.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,13.0,8.0,21.0,14.0,16.0,533.0,467.0,2.0,24.0,13.0
172,19,2020-10-19,Universitario,Cusco FC,3,2,https://www.ceroacero.es/match.php?id=7324820,4,8,10,23,1,1,15,18,5,4,56 %,44 %,Universitario vs Cusco FC,3.0,0.0,3.0,5.0,1.0,0.0,0.0,0.0,10.0,4.0,7.0,3.0,14.0,412.0,301.0,6.0,15.0,17.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,23.0,8.0,11.0,6.0,6.0,311.0,205.0,1.0,18.0,13.0
173,19,2020-10-19,Atlético Grau,Univ. Técnica,0,0,https://www.ceroacero.es/match.php?id=7324818,3,1,10,7,None,None,10,27,9,4,61 %,39 %,Atlético Grau vs Univ. Técnica,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,10.0,3.0,21.0,14.0,11.0,442.0,336.0,1.0,10.0,25.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,7.0,1.0,30.0,15.0,29.0,277.0,188.0,3.0,27.0,9.0
